In [13]:
import pdfplumber
import pandas as pd
import re

# -----------------------------
# 1. Define file paths
# -----------------------------
files = {
    "2020/2021": r"C:\Users\SYDNEY\Documents\moringa\phase5\Analysis-of-National-Government-Ministries-Departments-and-Agencies-Budget-Data\Data\NATIONAL-GOVERNMENT-AUDIT-REPORT-2020-2021.pdf",
    "2021/2022": r"C:\Users\SYDNEY\Documents\moringa\phase5\Analysis-of-National-Government-Ministries-Departments-and-Agencies-Budget-Data\Data\BLUEBOOK-2021-2022.pdf",
    "2022/2023": r"C:\Users\SYDNEY\Documents\moringa\phase5\Analysis-of-National-Government-Ministries-Departments-and-Agencies-Budget-Data\Data\Auditor-General-Report-on-National-Government-for-2022-2023.pdf",
    "2023/2024": r"C:\Users\SYDNEY\Documents\moringa\phase5\Analysis-of-National-Government-Ministries-Departments-and-Agencies-Budget-Data\Data\National-Government-MinistriesDepartments-And-Agencies-2023-2024.pdf"

}

# -----------------------------
# 2. Helpers
# -----------------------------
def clean_number(val):
    """Remove commas, Kshs, spaces; convert to float"""
    if not val:
        return None
    val = re.sub(r"[^\d.]", "", val)
    try:
        return float(val)
    except:
        return None

def is_valid_ministry(name):
    """Filter out obvious noise lines"""
    invalid_patterns = ["Total", "REFERENCE", "NO.", "Loan", "Purchase of", "Compensation of", "CKE"]
    for pat in invalid_patterns:
        if pat.lower() in name.lower():
            return False
    return True

# -----------------------------
# 3. Extraction function (scan full doc)
# -----------------------------
def extract_budget_full(file_path, year):
    rows = []
    with pdfplumber.open(file_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if not text:
                continue
            for line in text.split("\n"):
                # Regex: ministry name + 2 numbers
                match = re.match(r"(.+?)\s+([\d,]+)\s+([\d,]+)", line)
                if match:
                    ministry = match.group(1).strip()
                    approved = clean_number(match.group(2))
                    actual = clean_number(match.group(3))
                    if approved and actual and is_valid_ministry(ministry):
                        variance = approved - actual
                        rows.append({
                            "Ministry/Department/Agency": ministry,
                            "Financial Year": year,
                            "Approved Budget": approved,
                            "Actual Expenditure": actual,
                            "Variance": variance
                        })
    return pd.DataFrame(rows)

# -----------------------------
# 4. Run across all years
# -----------------------------
dfs = []
for year, path in files.items():
    print(f"Extracting {year} from {path} ...")
    dfs.append(extract_budget_full(path, year))

final_df = pd.concat(dfs, ignore_index=True)

# -----------------------------
# 5. Save final CSV
# -----------------------------
output_path = r"C:\Users\SYDNEY\Documents\moringa\phase5\Analysis-of-National-Government-Ministries-Departments-and-Agencies-Budget-Data\Data\Kenya_National_Govt_Budget_2021_2024.csv"
final_df.to_csv(output_path, index=False)

print("Full extraction complete. Rows:", len(final_df))
print("CSV saved at:", output_path)
print(final_df.head(20))


Extracting 2020/2021 from C:\Users\SYDNEY\Documents\moringa\phase5\Analysis-of-National-Government-Ministries-Departments-and-Agencies-Budget-Data\Data\NATIONAL-GOVERNMENT-AUDIT-REPORT-2020-2021.pdf ...
Extracting 2021/2022 from C:\Users\SYDNEY\Documents\moringa\phase5\Analysis-of-National-Government-Ministries-Departments-and-Agencies-Budget-Data\Data\BLUEBOOK-2021-2022.pdf ...
Extracting 2022/2023 from C:\Users\SYDNEY\Documents\moringa\phase5\Analysis-of-National-Government-Ministries-Departments-and-Agencies-Budget-Data\Data\Auditor-General-Report-on-National-Government-for-2022-2023.pdf ...
Extracting 2023/2024 from C:\Users\SYDNEY\Documents\moringa\phase5\Analysis-of-National-Government-Ministries-Departments-and-Agencies-Budget-Data\Data\National-Government-MinistriesDepartments-And-Agencies-2023-2024.pdf ...
Full extraction complete. Rows: 561
CSV saved at: C:\Users\SYDNEY\Documents\moringa\phase5\Analysis-of-National-Government-Ministries-Departments-and-Agencies-Budget-Data\Da